In [1]:
import inspect
from typing import ClassVar, Dict
from numpydoc.docscrape import NumpyDocString
import json


class BaseCalculator:
    """Base class for type hinting a property that is calculated with a calculator."""
    ui_type: str = "calculator"
    annotation: object 
    ui_name: str

    

def inspectObject(cls):
    """
    Retrun a dict of the arguments of a supercharged class.

    Parameters
    ----------
    cls
        un-initialised class.
    key
        Key to get the specified class from the registry.
    none_value
        Value for undifined values.
    """

    none_value = None
    mapping = {
        inspect._empty: "null",
        str: "string",
        int: "integer",
        float: "number",
        dict: "object",
        bool: "boolean",
        list: "array",
        None: "null",

    }
    new_args = {}
    Object = cls
    required_properties = []

    for c in inspect.getmro(Object):
        args = dict(inspect.signature(c).parameters)
        doc = NumpyDocString(c.__doc__) if c.__doc__ else NumpyDocString("")
        parameters = {
            P.name: {"description": P.desc, "unit": P.type}
            for P in doc["Parameters"]
        }

        for arg in args:                
            unit = parameters.get(arg, {}).get("unit", none_value)
            descr = parameters.get(arg, {}).get("description", [])
            description = ""
            for d in descr:
                description += d

            annotation = args[arg].annotation    

            ui_type = none_value
            ui_name = none_value
            if issubclass(annotation, BaseCalculator):
                ui_type = annotation.ui_type
                ui_name = annotation.ui_name
                annotation = annotation.annotation

            annotation = mapping.get(annotation, none_value)

            default = (
                args[arg].default
                if args[arg].default is not inspect._empty
                else none_value
            )
            if args[arg].default is inspect._empty:
                required_properties.append(arg)

            new_args[arg] = {}

            new_args[arg]["parentclass"] = c.__str__(c)
            if ui_type and ui_name:
                new_args[arg][f"ui:{ui_type}"] = ui_name
            if annotation:
                new_args[arg]["type"] = annotation
            if default:
                new_args[arg]["default"] = default
            if description:
                new_args[arg]["description"] = description
            if unit:
                new_args[arg]["unit"] = unit

    del new_args["args"]
    del new_args["kwargs"]

    required_properties = list(set(required_properties))
    if "args" in required_properties:
        required_properties.remove("args")
    if "kwargs" in required_properties:
        required_properties.remove("kwargs")

    json_shema = {
        "type": "object",
        "properties": new_args,
        "required":required_properties
    }

    return json_shema


In [3]:
class ExampleCalculator(BaseCalculator):
    """This class can be used for type hinting a property that is calculated with a calculator."""
    annotation = float
    ui_type = "calculator"
    ui_name = "exampleCalculator"


class B:
    """
    Hallo, Dit is een testclass.

    Parameters
    ----------
    b : m/s
        velocity of the vessel in m/s
    """
    def __init__(self, b: str, f, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.b = b
        
        
class C:
    """
    Hallo, Dit is een testclass.

    Parameters
    ----------
    c : degrees
        description: heading of the 
        vessel in degrees.
    d : sjaakies / olifant
        random interger.
    e
        speed of light.
    """
    def __init__(self, e:ExampleCalculator,a=None, c:int = None, d:dict={"A":1}, *args, **kwargs):       
        super().__init__(*args, **kwargs)
        self.c = c
        
class D:
    def my_new_function():
        pass

        
class ABC(B,C, D):
    key = "test"

In [4]:
print(json.dumps(inspectObject(ABC), indent = 4))

{
    "type": "object",
    "properties": {
        "b": {
            "parentclass": "<class '__main__.B'>",
            "type": "string",
            "description": "velocity of the vessel in m/s",
            "unit": "m/s"
        },
        "f": {
            "parentclass": "<class '__main__.B'>",
            "type": "null"
        },
        "e": {
            "parentclass": "<class '__main__.C'>",
            "ui:calculator": "exampleCalculator",
            "type": "number",
            "description": "speed of light."
        },
        "a": {
            "parentclass": "<class '__main__.C'>",
            "type": "null"
        },
        "c": {
            "parentclass": "<class '__main__.C'>",
            "type": "integer",
            "description": "description: heading of the vessel in degrees.",
            "unit": "degrees"
        },
        "d": {
            "parentclass": "<class '__main__.C'>",
            "type": "object",
            "default": {
                "